In [1]:
import pandas as pd
import time
import glob

In [7]:
df = pd.read_pickle("../workproduct-files/batchRuns/batch142_1375-1379.pkl")

In [8]:
df

,searchTerms,wikipediaSearchSuccessful,findQuestionCategories_meta,wikipediaArticleTitle,wikipediaArticleID,categoryPath,parentCategories
1375,"[goose, france]",False,"[Database call timed out (90 seconds), Databas...",None,NaN,None,None
1376,[steak],True,[SUCCESS],Steak,34931824.0,pages.title pages.id inters...,pages.title pa...
1377,"[tartare, steak]",True,"[Database call timed out (90 seconds), SUCCESS]",Steak,34931824.0,pages.title pages.id inters...,pages.title pa...
1378,"[jeroboam, champagne, bottle]",True,"[Database call timed out (90 seconds), SUCCESS]",Champagne,53754.0,pages.title pages.id inter...,pages...
1379,"[Nebuchadnezzar, nebuchadnezzar, champagne, bo...",True,"[Database call timed out (90 seconds), SUCCESS]",Nebuchadnezzar_II,52346.0,pages.title pages.id inter...,pages...


In [9]:
RunMeta = pd.read_csv("../workproduct-files" + "/batchRuns.csv", delimiter=";")

In [10]:
RunMeta

,startIndex,stopIndex,startTime,endTime,runTime
0,-1,-1,NaN,NaN,NaN
1,0,9,2020-10-03 15:57:52,2020-10-03 16:02:54,302.0
2,10,19,2020-10-03 16:02:54,2020-10-03 16:04:33,99.0
3,20,29,2020-10-03 16:04:33,2020-10-03 16:10:09,336.0
4,30,39,2020-10-03 16:10:09,2020-10-03 16:12:35,146.0
...,...,...,...,...,...
279,2060,2064,2020-10-04 16:00:10,2020-10-04 16:04:59,289.0
280,2065,2069,2020-10-04 16:04:59,2020-10-04 16:08:12,193.0
281,2070,2074,2020-10-04 16:08:12,2020-10-04 16:10:06,114.0
282,2075,2079,2020-10-04 16:10:06,2020-10-04 16:10:30,24.0


In [11]:
Run_time = RunMeta.iloc[:,-1].sum(skipna = True)

In [16]:
Run_time/2084*65000/3600/24

11.917547495912418

In [18]:
Run_time/3600

9.170277777777779

In [ ]:
Run5 = "../workproduct-files/batchRuns/Run5"
Run6 = "../workproduct-files/batchRuns/Run6"

In [ ]:
Run5Meta = pd.read_csv(Run5 + "/batchRuns.csv", delimiter=";")
Run6Meta = pd.read_csv(Run6 + "/batchRuns.csv", delimiter=";")

In [ ]:
Run5_time = Run5Meta.iloc[:,-1].sum(skipna = True)

In [ ]:
Run6_time = Run6Meta.iloc[:,-1].sum(skipna = True)

In [ ]:
# Run5 and Run6 now comparable.
# Run6 still jammed for some reason
# --> compare run times (above) and failure counts
# Read all pickle files into separate dfs for both runs

In [ ]:
Run5_files = glob.glob(Run5 + "/pickles/*.pkl")
Run6_files = glob.glob(Run6 + "/pickles/*.pkl")

In [ ]:
Run5DF = pd.DataFrame()
for filename in Run5_files:
    Run5DF = Run5DF.append(pd.read_pickle(filename))
Run5DF = Run5DF.sort_index()

In [ ]:
Run6DF = pd.DataFrame()
for filename in Run6_files:
    Run6DF = Run6DF.append(pd.read_pickle(filename))
Run6DF = Run6DF.sort_index()

In [ ]:
Run5_wikisearchFailed = len(Run5DF[Run5DF.wikipediaSearchSuccessful == False])

In [ ]:
Run6_wikisearchFailed = len(Run6DF[Run6DF.wikipediaSearchSuccessful == False])

In [ ]:
def successIndex(a):
    try:
        result = a.index("SUCCESS")
    except ValueError:
        result = -1
    
    return result

In [ ]:
Run5DF["searchTermUsed"] = Run5DF["findQuestionCategories_meta"].apply(lambda x: successIndex(x))

In [ ]:
Run6DF["searchTermUsed"] = Run6DF["findQuestionCategories_meta"].apply(lambda x: successIndex(x))

In [ ]:
termComparison = pd.DataFrame
termComparison = pd.concat([Run5DF["searchTermUsed"], Run6DF["searchTermUsed"]], axis=1)
termComparison.columns = ['a', 'b']
differentTermsCount = len(termComparison.loc[termComparison['a'] != termComparison['b']])

In [ ]:
print("Run 5:")
print("Run time: " + str(Run5_time))
print("Search failed: " + str(Run5_wikisearchFailed))
print("")
print("Run 6:")
print("Run time: " + str(Run6_time))
print("Search failed: " + str(Run6_wikisearchFailed))
print("")
print("Run 6 / Run 5")
print("Run time: " + str(Run6_time/Run5_time))
print("Search failed: " + str(Run6_wikisearchFailed/Run5_wikisearchFailed))
print("Term changed (abs): " + str(differentTermsCount))
print("Term changed (rel): " + str(differentTermsCount / len(termComparison)))

In [ ]:
# Estimated runtime in days for full dataset of 65k rows
Run6_time/len(termComparison)*65000/3600/24